<p> Notice: This notebook is not optimized for memory nor performance yet. Please use it with caution when handling large datasets.


### Notice: Please ignore Feature engineering part if you are using a ready dataset

# Feature engineering

This notebook is for BDSE12_03G_HomeCredit_V2.csv processing for bear LGBM final

adding le for categorical and reciprocal for numerical columns

### Prepare work environment

In [1]:
# Pandas for managing datasets
import numpy as np
import pandas as pd

In [2]:
np.__version__, pd.__version__

('1.17.4', '0.25.3')

In [3]:
# math for operating numbers
import math

In [4]:
import gc

In [5]:
# Change pd displayg format for float
pd.options.display.float_format = '{:,.8f}'.format

In [6]:
# Matplotlib for additional customization
from matplotlib import pyplot as plt
%matplotlib inline

In [7]:
# Seaborn for plotting and styling
import seaborn as sns
#Seaborn set() to set aesthetic parameters in one step.
sns.set() 

---

### Read & combine datasets

In [8]:
appl_all_df = pd.read_csv('../../datasets/homecdt_fteng/BDSE12_03G_HomeCredit_V2.csv',index_col=0)

In [9]:
appl_all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 797 entries, AMT_ANNUITY to GOODS_PRICE_PREV%
dtypes: float64(741), int64(42), object(14)
memory usage: 2.1+ GB


---

In [10]:
# appl_all_df.apply(lambda x:x.unique().size).describe()

In [11]:
appl_all_df['TARGET'].unique(), \
appl_all_df['TARGET'].unique().size

(array([ 1.,  0., nan]), 3)

In [12]:
appl_all_df['TARGET'].value_counts()

0.00000000    282686
1.00000000     24825
Name: TARGET, dtype: int64

In [13]:
appl_all_df['TARGET'].isnull().sum(), \
appl_all_df['TARGET'].size, \
(appl_all_df['TARGET'].isnull().sum()/appl_all_df['TARGET'].size).round(4)

(48744, 356255, 0.1368)

In [14]:
# Make sure we can use the nullness of 'TARGET' column to separate train & test
# assert appl_all_df['TARGET'].isnull().sum() == appl_test_df.shape[0]

---

## Randomized sampleing:

#### If the dataset is too large, consider following randomized sampling from original dataset to facilitate development and testing

In [15]:
# Randomized sampling from original dataset.
# This is just for simplifying the development process
# After coding is complete, should replace all df-->df, and remove this cell
# Reference: https://yiidtw.github.io/blog/2018-05-29-how-to-shuffle-dataframe-in-pandas/

# df= appl_all_df.sample(n = 1000).reset_index(drop=True)
# df.shape

In [16]:
# df.head()

---

---

## multiprocessing (beta)

In [17]:
# ToRead: multi-threading
# http://violin-tao.blogspot.com/2017/05/python3_26.html
# https://medium.com/@peilee_98185/%E6%94%BE%E9%96%8B%E9%82%A3%E8%A8%98%E6%86%B6%E9%AB%94-%E4%B9%8B-python-%E8%99%95%E7%90%86%E5%A4%A7%E8%B3%87%E6%96%99-84fd41806694
import multiprocessing as mp

In [18]:
# pool = mp.Pool(4) # 裡面填要開幾核心

In [19]:
# result = pool.map(data_process_function, dfs )
# # dfs 為裝了很多 Pandas DataFrame 的 list

---

## Tool: Get numerical/ categorical variables(columns) from a dataframe

In [20]:
def get_num_df (data_df, unique_value_threshold: int):
    """ 
    Output: a new dataframe with columns of numerical variables from the input dataframe.
    Input: 
        data_df: original dataframe, 
        unique_value_threshold(int): number of unique values of each column
    e.g. If we define a column with > 3 unique values as being numerical variable, unique_value_threshold = 3
    """
    num_mask = data_df.apply(lambda x:x.unique().size > unique_value_threshold,axis=0) 
    num_df = data_df[data_df.columns[num_mask]]
    return num_df

def get_cat_df (data_df, unique_value_threshold: int):
    """ 
    Output: a new dataframe with columns of categorical variables from the input dataframe.
    Input: 
        data_df: original dataframe, 
        unique_value_threshold(int): number of unique values of each column
    e.g. If we define a column with =<3 unique values as being numerical variable, unique_value_threshold = 3
    """
    cat_mask = data_df.apply(lambda x:x.unique().size <= unique_value_threshold,axis=0) 
    cat_df = data_df[data_df.columns[cat_mask]]
    return cat_df


In [21]:
# Be careful when doing this assertion with large datasets
# assert get_cat_df(appl_all_df, 3).columns.size + get_num_df(appl_all_df, 3).columns.size == appl_all_df.columns.size

---

#### Splitting id_target_df, cat_df, num_df

In [22]:
# Separate id and target columns before any further processing
id_target_df = appl_all_df.loc[:, ['SK_ID_CURR','TARGET']]

# Get the operating appl_all_df by removing id and target columns
appl_all_df_opr = appl_all_df.drop(['SK_ID_CURR','TARGET'], axis=1)

# A quick check of their shapes
appl_all_df.shape, id_target_df.shape, appl_all_df_opr.shape

((356255, 797), (356255, 2), (356255, 795))

In [23]:
# Spliting the numerical and categorical variable containing columns via the tools decribed above.
# Max identified unique value of categorical column 'ORGANIZATION_TYPE' = 58
cat_df = get_cat_df (appl_all_df_opr, 58)
num_df = get_num_df (appl_all_df_opr, 58)

In [24]:
cat_df.info()
num_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 250 entries, AMT_REQ_CREDIT_BUREAU_DAY to AMT_REQ_CREDIT_BUREAU_MON/QRT
dtypes: float64(198), int64(38), object(14)
memory usage: 682.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 545 entries, AMT_ANNUITY to GOODS_PRICE_PREV%
dtypes: float64(542), int64(3)
memory usage: 1.4 GB


In [25]:
# A quick check of their shapes
appl_all_df_opr.shape, cat_df.shape, num_df.shape

((356255, 795), (356255, 250), (356255, 545))

In [26]:
assert cat_df.shape[1] + num_df.shape[1] + id_target_df.shape[1] \
    == appl_all_df_opr.shape[1] + id_target_df.shape[1] \
    == appl_all_df.shape[1]

assert cat_df.shape[0] == num_df.shape[0] == id_target_df.shape[0] \
    == appl_all_df_opr.shape[0] \
    == appl_all_df.shape[0]

In [27]:
# Apply the following gc if memory is running slow
appl_all_df_opr.info()
appl_all_df.info()
del appl_all_df_opr
del appl_all_df
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 795 entries, AMT_ANNUITY to GOODS_PRICE_PREV%
dtypes: float64(740), int64(41), object(14)
memory usage: 2.1+ GB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 797 entries, AMT_ANNUITY to GOODS_PRICE_PREV%
dtypes: float64(741), int64(42), object(14)
memory usage: 2.1+ GB


0

---

## Dealing with categorical variables

#### Transform to String (i.e., python object) and fill nan with String 'nan'

In [28]:
cat_df_obj = cat_df.astype(str)

In [29]:
assert np.all(cat_df_obj.dtypes) == object

# There are no NA left
assert all(cat_df_obj.isnull().sum())==0

In [30]:
# The float nan will be tranformed to String 'nan'
# Use this assertion carefully when dealing with extra-large datasets
assert cat_df.isnull().equals(cat_df_obj.isin({'nan'}))

#### Dealing with special columns

Replace 'nan' with 'not specified' in column 'FONDKAPREMONT_MODE'

In [31]:
# Do the replacement and re-assign the modified column back to the original dataframe
cat_df_obj['FONDKAPREMONT_MODE'] = cat_df_obj['FONDKAPREMONT_MODE'].replace('nan','not specified')

In [32]:
# check again the unique value, it should be 1 less than the original cat_df
assert cat_df['FONDKAPREMONT_MODE'].unique().size == cat_df_obj['FONDKAPREMONT_MODE'].unique().size +1

In [33]:
# Apply the following gc if memory is running slow
cat_df.info()
del cat_df
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 250 entries, AMT_REQ_CREDIT_BUREAU_DAY to AMT_REQ_CREDIT_BUREAU_MON/QRT
dtypes: float64(198), int64(38), object(14)
memory usage: 682.2+ MB


132

---

#### Do label encoding

In [34]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()

In [35]:
# apply le on categorical feature columns
cat_df_obj_le = cat_df_obj[:].apply(lambda col: le.fit_transform(col))

In [36]:
cat_df_obj_le

,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,DEF_30_CNT_SOCIAL_CIRCLE,...,AMT_REQ_CREDIT_BUREAU_HOUR/QRT,AMT_REQ_CREDIT_BUREAU_HOUR/YEAR,AMT_REQ_CREDIT_BUREAU_DAY/WEEK,AMT_REQ_CREDIT_BUREAU_DAY/MON,AMT_REQ_CREDIT_BUREAU_DAY/QRT,AMT_REQ_CREDIT_BUREAU_DAY/YEAR,AMT_REQ_CREDIT_BUREAU_WEEK/MON,AMT_REQ_CREDIT_BUREAU_WEEK/QRT,AMT_REQ_CREDIT_BUREAU_WEEK/YEAR,AMT_REQ_CREDIT_BUREAU_MON/QRT
0,0,0,0,0,0,1,0,0,1,2,...,9,0,16,16,12,0,25,15,0,46
1,0,0,0,0,0,0,0,8,0,0,...,9,20,16,16,12,30,25,15,39,46
2,0,0,0,0,0,0,0,0,1,0,...,9,20,16,16,12,30,25,15,39,46
3,9,5,24,11,9,25,0,8,0,0,...,9,20,16,16,12,30,25,15,39,46
4,0,0,0,0,0,0,0,0,1,0,...,9,20,16,16,12,30,25,15,39,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356250,0,0,0,0,0,1,0,0,0,0,...,9,0,16,16,12,0,25,15,0,46
356251,9,5,24,11,9,25,7,12,0,0,...,9,20,16,16,12,30,25,15,39,46
356252,0,0,0,5,0,1,1,11,0,0,...,0,0,16,16,0,0,25,0,0,0
356253,0,0,0,0,0,12,0,8,1,0,...,9,0,16,16,12,0,25,15,0,46


In [37]:
cat_df_obj_le.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 250 entries, AMT_REQ_CREDIT_BUREAU_DAY to AMT_REQ_CREDIT_BUREAU_MON/QRT
dtypes: int32(250)
memory usage: 342.5 MB


In [38]:
# assert all(cat_df_obj_le == cat_df_obj)
assert cat_df_obj_le.shape[1] == cat_df_obj.shape[1]

In [39]:
del cat_df_obj
gc.collect()

44

#### Do one-hot encoding

Check the input dataframe (i.e., cat_df_obj)

In [40]:
# cat_df_obj.shape

In [41]:
# cat_df_obj.apply(lambda x:x.unique().size).sum()

In [42]:
# ?pd.get_dummies

In [43]:
# # pd.get_dummies() method deals only with categorical variables.
# # Although it has a built-in argument 'dummy_na' to manage the na value, 
# # our na value has already been converted to string object which are not recognized by the method.
# # Let's just move forward as planned
# cat_df_obj_ohe = pd.get_dummies(cat_df_obj, drop_first=True)
# cat_df_obj_ohe.shape

In [44]:
# # Make sure the ohe is successful
# assert np.all(np.isin(cat_df_obj_ohe.values,[0,1])) == True
# # cat_df_obj_ohe.dtypes
# assert np.all(cat_df_obj_ohe.dtypes) == 'uint8'
# # make sure the column counts are correct
# assert cat_df_obj.apply(lambda x:x.unique().size).sum() == cat_df_obj_ohe.shape[1] + cat_df_obj.shape[1]

In [45]:
# cat_df_obj_ohe.info()

In [46]:
# # Apply the following gc if memory is running slow
# del cat_df_obj
# gc.collect()

In [47]:
# %timeit np.isin(cat_df_obj_ohe.values,[0,1])
# # 1.86 s ± 133 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# %timeit cat_df_obj_ohe.isin([0 , 1])
# # 3.38 s ± 32.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [48]:
# %timeit np.all(np.isin(cat_df_obj_ohe.values,[0,1]))
# # 1.85 s ± 28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# %timeit np.all(cat_df_obj_ohe.isin([0 , 1]))
# # 3.47 s ± 193 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

---

## Dealing with numerial variables

#### Make reciprocals

In [49]:
num_df.shape

(356255, 545)

In [50]:
num_df

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,...,AMT_REQ_CREDIT_BUREAU_QRT/YEAR,OUTSTANDING/CREDIT,OUTSTANDING/ANNUITY,PAYMENT_ALL_mean/CREDIT,ANNUITY_BU%,CREDIT%,ANNUITY/CREDIT%,ANNUITY_PREV%,CREDIT_PREV%,GOODS_PRICE_PREV%
0,"24,700.50000000","406,597.50000000","351,000.00000000","202,500.00000000",0.02470000,0.02500000,0.02520000,0.03690000,0.03690000,0.03830000,...,0.00000000,0.00000000,0.00000000,0.54015506,inf,0.90355000,inf,inf,inf,inf
1,"35,698.50000000","1,293,502.50000000","1,129,500.00000000","270,000.00000000",0.09590000,0.09680000,0.09240000,0.05290000,0.05290000,0.05380000,...,nan,0.00000000,0.00000000,0.41717859,nan,1.61377502,nan,0.31610227,1.26618404,1.27610999
2,"6,750.00000000","135,000.00000000","135,000.00000000","67,500.00000000",nan,nan,nan,nan,nan,nan,...,nan,0.00000000,0.00000000,0.15769233,nan,"1,071.42857143",nan,inf,inf,inf
3,"29,686.50000000","312,682.50000000","297,000.00000000","135,000.00000000",nan,nan,nan,nan,nan,nan,...,nan,0.00000000,0.00000000,1.07366995,nan,nan,nan,0.72597880,0.34489006,0.40824168
4,"21,865.50000000","513,000.00000000","513,000.00000000","121,500.00000000",nan,nan,nan,nan,nan,nan,...,nan,0.00000000,0.00000000,0.31427991,nan,inf,nan,0.96098322,1.84734454,2.15272649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356250,"17,473.50000000","412,560.00000000","270,000.00000000","121,500.00000000",nan,nan,nan,nan,nan,nan,...,0.00000000,0.00000000,0.00000000,0.66198702,1.01251630,0.83936435,1.18109181,inf,inf,inf
356251,"31,909.50000000","622,413.00000000","495,000.00000000","157,500.00000000",nan,nan,nan,nan,nan,nan,...,nan,0.00000000,0.00000000,0.22786947,nan,nan,nan,2.58802315,3.27148471,2.98841532
356252,"33,205.50000000","315,000.00000000","315,000.00000000","202,500.00000000",0.11130000,0.11240000,0.11340000,0.13640000,0.13640000,0.14150000,...,3.00000000,0.00000000,0.00000000,0.29408243,6.57079252,0.14278048,1.05441320,6.59066326,2.04894041,3.42194392
356253,"25,128.00000000","450,000.00000000","450,000.00000000","225,000.00000000",0.16290000,0.16450000,0.16600000,0.07230000,0.07230000,0.07500000,...,0.00000000,0.00000000,0.00000000,0.55645310,0.91271657,0.45400016,0.60260820,1.09348180,0.99146350,1.13660517


In [51]:
num_reciprocals_df = num_df.rdiv(1).add_suffix('_recip')

In [52]:
num_reciprocals_df.shape

(356255, 545)

In [53]:
num_reciprocals_df

,AMT_ANNUITY_recip,AMT_CREDIT_recip,AMT_GOODS_PRICE_recip,AMT_INCOME_TOTAL_recip,APARTMENTS_AVG_recip,APARTMENTS_MEDI_recip,APARTMENTS_MODE_recip,BASEMENTAREA_AVG_recip,BASEMENTAREA_MEDI_recip,BASEMENTAREA_MODE_recip,...,AMT_REQ_CREDIT_BUREAU_QRT/YEAR_recip,OUTSTANDING/CREDIT_recip,OUTSTANDING/ANNUITY_recip,PAYMENT_ALL_mean/CREDIT_recip,ANNUITY_BU%_recip,CREDIT%_recip,ANNUITY/CREDIT%_recip,ANNUITY_PREV%_recip,CREDIT_PREV%_recip,GOODS_PRICE_PREV%_recip
0,0.00004049,0.00000246,0.00000285,0.00000494,40.48582996,40.00000000,39.68253968,27.10027100,27.10027100,26.10966057,...,inf,inf,inf,1.85132027,0.00000000,1.10674561,0.00000000,0.00000000,0.00000000,0.00000000
1,0.00002801,0.00000077,0.00000089,0.00000370,10.42752868,10.33057851,10.82251082,18.90359168,18.90359168,18.58736059,...,nan,inf,inf,2.39705494,nan,0.61966506,nan,3.16353316,0.78977461,0.78363151
2,0.00014815,0.00000741,0.00000741,0.00001481,nan,nan,nan,nan,nan,nan,...,nan,inf,inf,6.34146238,nan,0.00093333,nan,0.00000000,0.00000000,0.00000000
3,0.00003369,0.00000320,0.00000337,0.00000741,nan,nan,nan,nan,nan,nan,...,nan,inf,inf,0.93138492,nan,nan,nan,1.37745069,2.89947471,2.44952939
4,0.00004573,0.00000195,0.00000195,0.00000823,nan,nan,nan,nan,nan,nan,...,nan,inf,inf,3.18187692,nan,0.00000000,nan,1.04060089,0.54131754,0.46452720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356250,0.00005723,0.00000242,0.00000370,0.00000823,nan,nan,nan,nan,nan,nan,...,inf,inf,inf,1.51060364,0.98763842,1.19137774,0.84667423,0.00000000,0.00000000,0.00000000
356251,0.00003134,0.00000161,0.00000202,0.00000635,nan,nan,nan,nan,nan,nan,...,nan,inf,inf,4.38847737,nan,nan,nan,0.38639531,0.30567161,0.33462551
356252,0.00003012,0.00000317,0.00000317,0.00000494,8.98472597,8.89679715,8.81834215,7.33137830,7.33137830,7.06713781,...,0.33333333,inf,inf,3.40040717,0.15218864,7.00375852,0.94839481,0.15172980,0.48805714,0.29223156
356253,0.00003980,0.00000222,0.00000222,0.00000444,6.13873542,6.07902736,6.02409639,13.83125864,13.83125864,13.33333333,...,inf,inf,inf,1.79709665,1.09563037,2.20264240,1.65945302,0.91450997,1.00861000,0.87981299


In [54]:
# Combine original num_df with num_reciprocals_df as a new num_df (to fit in the previous code)
num_df = pd.concat([num_df, num_reciprocals_df], axis = 'columns')

In [55]:
num_df.shape

(356255, 1090)

In [56]:
num_df

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,...,AMT_REQ_CREDIT_BUREAU_QRT/YEAR_recip,OUTSTANDING/CREDIT_recip,OUTSTANDING/ANNUITY_recip,PAYMENT_ALL_mean/CREDIT_recip,ANNUITY_BU%_recip,CREDIT%_recip,ANNUITY/CREDIT%_recip,ANNUITY_PREV%_recip,CREDIT_PREV%_recip,GOODS_PRICE_PREV%_recip
0,"24,700.50000000","406,597.50000000","351,000.00000000","202,500.00000000",0.02470000,0.02500000,0.02520000,0.03690000,0.03690000,0.03830000,...,inf,inf,inf,1.85132027,0.00000000,1.10674561,0.00000000,0.00000000,0.00000000,0.00000000
1,"35,698.50000000","1,293,502.50000000","1,129,500.00000000","270,000.00000000",0.09590000,0.09680000,0.09240000,0.05290000,0.05290000,0.05380000,...,nan,inf,inf,2.39705494,nan,0.61966506,nan,3.16353316,0.78977461,0.78363151
2,"6,750.00000000","135,000.00000000","135,000.00000000","67,500.00000000",nan,nan,nan,nan,nan,nan,...,nan,inf,inf,6.34146238,nan,0.00093333,nan,0.00000000,0.00000000,0.00000000
3,"29,686.50000000","312,682.50000000","297,000.00000000","135,000.00000000",nan,nan,nan,nan,nan,nan,...,nan,inf,inf,0.93138492,nan,nan,nan,1.37745069,2.89947471,2.44952939
4,"21,865.50000000","513,000.00000000","513,000.00000000","121,500.00000000",nan,nan,nan,nan,nan,nan,...,nan,inf,inf,3.18187692,nan,0.00000000,nan,1.04060089,0.54131754,0.46452720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356250,"17,473.50000000","412,560.00000000","270,000.00000000","121,500.00000000",nan,nan,nan,nan,nan,nan,...,inf,inf,inf,1.51060364,0.98763842,1.19137774,0.84667423,0.00000000,0.00000000,0.00000000
356251,"31,909.50000000","622,413.00000000","495,000.00000000","157,500.00000000",nan,nan,nan,nan,nan,nan,...,nan,inf,inf,4.38847737,nan,nan,nan,0.38639531,0.30567161,0.33462551
356252,"33,205.50000000","315,000.00000000","315,000.00000000","202,500.00000000",0.11130000,0.11240000,0.11340000,0.13640000,0.13640000,0.14150000,...,0.33333333,inf,inf,3.40040717,0.15218864,7.00375852,0.94839481,0.15172980,0.48805714,0.29223156
356253,"25,128.00000000","450,000.00000000","450,000.00000000","225,000.00000000",0.16290000,0.16450000,0.16600000,0.07230000,0.07230000,0.07500000,...,inf,inf,inf,1.79709665,1.09563037,2.20264240,1.65945302,0.91450997,1.00861000,0.87981299


In [57]:
num_reciprocals_df.info()
del num_reciprocals_df
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 545 entries, AMT_ANNUITY_recip to GOODS_PRICE_PREV%_recip
dtypes: float64(545)
memory usage: 1.4 GB


0

#### Get na flags

In [58]:
# Consider np.inf as nan
# ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html
pd.options.mode.use_inf_as_na = True

In [59]:
num_df.shape

(356255, 1090)

In [60]:
# How many columns contain na value.
num_df.isna().any().sum()

1062

In [61]:
num_isna_df = num_df[num_df.columns[num_df.isna().any()]]
num_notna_df = num_df[num_df.columns[num_df.notna().all()]]

assert num_isna_df.shape[1] + num_notna_df.shape[1] == num_df.shape[1]
assert num_isna_df.shape[0] == num_notna_df.shape[0] == num_df.shape[0]

In [62]:
num_isna_df.shape, num_notna_df.shape

((356255, 1062), (356255, 28))

In [63]:
# num_df.isna().any(): column names for those na containing columns
# use it to transform values bool to int, and then add suffix on the column names to get the na-flag df
num_naFlag_df = num_isna_df.isna().astype(np.uint8).add_suffix('_na')
num_naFlag_df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 1062 entries, APARTMENTS_AVG_na to GOODS_PRICE_PREV%_recip_na
dtypes: uint8(1062)
memory usage: 363.5 MB


In [64]:
num_naFlag_df

,APARTMENTS_AVG_na,APARTMENTS_MEDI_na,APARTMENTS_MODE_na,BASEMENTAREA_AVG_na,BASEMENTAREA_MEDI_na,BASEMENTAREA_MODE_na,COMMONAREA_AVG_na,COMMONAREA_MEDI_na,COMMONAREA_MODE_na,ELEVATORS_AVG_na,...,AMT_REQ_CREDIT_BUREAU_QRT/YEAR_recip_na,OUTSTANDING/CREDIT_recip_na,OUTSTANDING/ANNUITY_recip_na,PAYMENT_ALL_mean/CREDIT_recip_na,ANNUITY_BU%_recip_na,CREDIT%_recip_na,ANNUITY/CREDIT%_recip_na,ANNUITY_PREV%_recip_na,CREDIT_PREV%_recip_na,GOODS_PRICE_PREV%_recip_na
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,0,1,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,0,1,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356250,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
356251,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,0,0,0
356252,0,0,0,0,0,0,1,1,1,0,...,0,1,1,0,0,0,0,0,0,0
356253,0,0,0,0,0,0,1,1,1,0,...,1,1,1,0,0,0,0,0,0,0


#### replace na with zero

In [65]:
num_isna_df = num_isna_df.fillna(0)
num_isna_df.shape

(356255, 1062)

In [66]:
# How many columns contain na value.
num_isna_df.isna().any().sum()

0

In [67]:
num_isna_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 1062 entries, APARTMENTS_AVG to GOODS_PRICE_PREV%_recip
dtypes: float64(1062)
memory usage: 2.8 GB


In [68]:
num_isna_df

,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,ELEVATORS_AVG,...,AMT_REQ_CREDIT_BUREAU_QRT/YEAR_recip,OUTSTANDING/CREDIT_recip,OUTSTANDING/ANNUITY_recip,PAYMENT_ALL_mean/CREDIT_recip,ANNUITY_BU%_recip,CREDIT%_recip,ANNUITY/CREDIT%_recip,ANNUITY_PREV%_recip,CREDIT_PREV%_recip,GOODS_PRICE_PREV%_recip
0,0.02470000,0.02500000,0.02520000,0.03690000,0.03690000,0.03830000,0.01430000,0.01440000,0.01440000,0.00000000,...,0.00000000,0.00000000,0.00000000,1.85132027,0.00000000,1.10674561,0.00000000,0.00000000,0.00000000,0.00000000
1,0.09590000,0.09680000,0.09240000,0.05290000,0.05290000,0.05380000,0.06050000,0.06080000,0.04970000,0.08000000,...,0.00000000,0.00000000,0.00000000,2.39705494,0.00000000,0.61966506,0.00000000,3.16353316,0.78977461,0.78363151
2,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.00000000,0.00000000,0.00000000,6.34146238,0.00000000,0.00093333,0.00000000,0.00000000,0.00000000,0.00000000
3,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.00000000,0.00000000,0.00000000,0.93138492,0.00000000,0.00000000,0.00000000,1.37745069,2.89947471,2.44952939
4,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.00000000,0.00000000,0.00000000,3.18187692,0.00000000,0.00000000,0.00000000,1.04060089,0.54131754,0.46452720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356250,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.00000000,0.00000000,0.00000000,1.51060364,0.98763842,1.19137774,0.84667423,0.00000000,0.00000000,0.00000000
356251,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.00000000,0.00000000,0.00000000,4.38847737,0.00000000,0.00000000,0.00000000,0.38639531,0.30567161,0.33462551
356252,0.11130000,0.11240000,0.11340000,0.13640000,0.13640000,0.14150000,0.00000000,0.00000000,0.00000000,0.16000000,...,0.33333333,0.00000000,0.00000000,3.40040717,0.15218864,7.00375852,0.94839481,0.15172980,0.48805714,0.29223156
356253,0.16290000,0.16450000,0.16600000,0.07230000,0.07230000,0.07500000,0.00000000,0.00000000,0.00000000,0.16000000,...,0.00000000,0.00000000,0.00000000,1.79709665,1.09563037,2.20264240,1.65945302,0.91450997,1.00861000,0.87981299


In [69]:
assert num_isna_df.shape == num_naFlag_df.shape

In [70]:
num_df = pd.concat([num_notna_df,num_isna_df,num_naFlag_df], axis = 'columns')

In [71]:
assert num_notna_df.shape[1] + num_isna_df.shape[1] + num_naFlag_df.shape[1] == num_df.shape[1]

In [72]:
num_df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 2152 entries, AMT_ANNUITY to GOODS_PRICE_PREV%_recip_na
dtypes: float64(1087), int64(3), uint8(1062)
memory usage: 3.2 GB


In [73]:
num_df

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,OWN_CAR_AGE,...,AMT_REQ_CREDIT_BUREAU_QRT/YEAR_recip_na,OUTSTANDING/CREDIT_recip_na,OUTSTANDING/ANNUITY_recip_na,PAYMENT_ALL_mean/CREDIT_recip_na,ANNUITY_BU%_recip_na,CREDIT%_recip_na,ANNUITY/CREDIT%_recip_na,ANNUITY_PREV%_recip_na,CREDIT_PREV%_recip_na,GOODS_PRICE_PREV%_recip_na
0,"24,700.50000000","406,597.50000000","351,000.00000000","202,500.00000000",-9461,-637,-2120,"-1,134.00000000","-3,648.00000000",0.00000000,...,1,1,1,0,0,0,0,0,0,0
1,"35,698.50000000","1,293,502.50000000","1,129,500.00000000","270,000.00000000",-16765,-1188,-291,-828.00000000,"-1,186.00000000",0.00000000,...,1,1,1,0,1,0,1,0,0,0
2,"6,750.00000000","135,000.00000000","135,000.00000000","67,500.00000000",-19046,-225,-2531,-815.00000000,"-4,260.00000000",26.00000000,...,1,1,1,0,1,0,1,0,0,0
3,"29,686.50000000","312,682.50000000","297,000.00000000","135,000.00000000",-19005,-3039,-2437,-617.00000000,"-9,833.00000000",0.00000000,...,1,1,1,0,1,1,1,0,0,0
4,"21,865.50000000","513,000.00000000","513,000.00000000","121,500.00000000",-19932,-3038,-3458,"-1,106.00000000","-4,311.00000000",0.00000000,...,1,1,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356250,"17,473.50000000","412,560.00000000","270,000.00000000","121,500.00000000",-19970,-5169,-3399,-684.00000000,"-9,094.00000000",0.00000000,...,1,1,1,0,0,0,0,0,0,0
356251,"31,909.50000000","622,413.00000000","495,000.00000000","157,500.00000000",-11186,-1149,-3003,0.00000000,"-3,015.00000000",0.00000000,...,1,1,1,0,1,1,1,0,0,0
356252,"33,205.50000000","315,000.00000000","315,000.00000000","202,500.00000000",-15922,-3037,-1504,-838.00000000,"-2,681.00000000",4.00000000,...,0,1,1,0,0,0,0,0,0,0
356253,"25,128.00000000","450,000.00000000","450,000.00000000","225,000.00000000",-13968,-2731,-1364,"-2,308.00000000","-1,461.00000000",0.00000000,...,1,1,1,0,0,0,0,0,0,0


In [138]:
# Apply the following gc if memory is running slow
del num_notna_df
del num_isna_df
del num_naFlag_df
gc.collect()

1820

#### Trying to decide if I should transform all float64 to float32 to save memory usage

In [77]:
# an IEEE 754 32-bit base-2 floating-point variable has a maximum value of (2 − 2^(−23)) × 2^(127) ≈ 3.4028235 × 10^(38)
# https://en.wikipedia.org/wiki/Single-precision_floating-point_format

In [74]:
num_df.max().max()

6.92442461481468e+17

In [75]:
num_df.min().min()

-1.9649999021612728e+18

In [78]:
# looks fine if converting to float 32

In [112]:
num_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 2152 entries, AMT_ANNUITY to GOODS_PRICE_PREV%_recip_na
dtypes: float64(1087), int64(3), uint8(1062)
memory usage: 3.2 GB


In [114]:
num_float64_df = num_df[num_df.columns[(num_df.dtypes == np.float64)]]
num_others_df = num_df[num_df.columns[(num_df.dtypes != np.float64)]]
num_float64_df.info()
print("")
num_others_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 1087 entries, AMT_ANNUITY to GOODS_PRICE_PREV%_recip
dtypes: float64(1087)
memory usage: 2.9 GB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 1065 entries, DAYS_BIRTH to GOODS_PRICE_PREV%_recip_na
dtypes: int64(3), uint8(1062)
memory usage: 371.7 MB


In [116]:
assert num_float64_df.shape[1] + num_others_df.shape[1] == num_df.shape[1]
assert num_float64_df.shape[0] == num_others_df.shape[0] == num_df.shape[0]

In [117]:
num_float32_df = num_float64_df.astype(np.float32)
num_float32_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 1087 entries, AMT_ANNUITY to GOODS_PRICE_PREV%_recip
dtypes: float32(1087)
memory usage: 1.4 GB


In [118]:
assert num_float32_df.shape == num_float64_df.shape

In [120]:
num_df = pd.concat([num_float32_df,num_others_df], axis = 'columns')
num_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 2152 entries, AMT_ANNUITY to GOODS_PRICE_PREV%_recip_na
dtypes: float32(1087), int64(3), uint8(1062)
memory usage: 1.8 GB


In [135]:
assert all(num_df.dtypes != np.float64)

In [136]:
# Apply the following gc if memory is running slow
del num_float64_df
del num_others_df
del num_float32_df
gc.collect()

NameError: name 'num_float64_df' is not defined

---

#### Normalization (DO LATER!!)

##### Generally, in tree-based models, the scale of the features does not matter.
https://scikit-learn.org/stable/modules/preprocessing.html#normalization
https://datascience.stackexchange.com/questions/22036/how-does-lightgbm-deal-with-value-scale

---

## Combine to a complete, processed dataset

In [139]:
frames = np.array([id_target_df, cat_df_obj_le, num_df])

In [140]:
id_target_df.shape, cat_df_obj_le.shape, num_df.shape

((356255, 2), (356255, 250), (356255, 2152))

In [141]:
appl_all_processed_df = pd.concat(frames, axis ='columns')
appl_all_processed_df.shape

(356255, 2404)

In [142]:
assert appl_all_processed_df.shape[1] == id_target_df.shape[1] + cat_df_obj_le.shape[1] + num_df.shape[1]

In [143]:
appl_all_processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356255 entries, 0 to 356254
Columns: 2404 entries, SK_ID_CURR to GOODS_PRICE_PREV%_recip_na
dtypes: float32(1087), float64(1), int32(250), int64(4), uint8(1062)
memory usage: 2.1 GB


In [144]:
# Apply the following gc if memory is running slow
del id_target_df
del cat_df_obj_le
del num_df
gc.collect()

22

---

## Export to CSV

In [ ]:
# # Export the dataframe to csv for future use
# appl_all_processed_df.to_csv('../../datasets/homecdt_fteng/ss_fteng_G3V2_le_recip_20200215a.csv', index = False)

In [ ]:
# # Export the dtypes Series to csv for future use
# appl_all_processed_df.dtypes.to_csv('../../datasets/homecdt_fteng/ss_fteng_G3V2_le_recip_20200215a_dtypes_series.csv', header = False)

---

---

Below not executed

## Balance the 'TARGET' column

In [ ]:
appl_all_processed_df['TARGET'].value_counts()

In [ ]:
balanceFactor = ((appl_all_processed_df['TARGET'].value_counts()[0])/(appl_all_processed_df['TARGET'].value_counts()[1])).round(0).astype(int)
balanceFactor
# appl_all_processed_df['TARGET'].value_counts()[0]
# appl_all_processed_df['TARGET'].value_counts()[1]

In [ ]:
default_df = appl_all_processed_df[appl_all_processed_df['TARGET']==1]
default_df.shape

In [ ]:
default_df_balanced = pd.concat( [default_df] * (balanceFactor - 1), sort=False, ignore_index=True )
default_df_balanced.shape

In [ ]:
appl_all_processed_df_balanced = pd.concat([appl_all_processed_df , default_df_balanced], sort=False, ignore_index=True)
appl_all_processed_df_balanced.shape

In [ ]:
(appl_all_processed_df_balanced['TARGET'].unique(),
(appl_all_processed_df_balanced['TARGET'].value_counts()[1], \
appl_all_processed_df_balanced['TARGET'].value_counts()[0], \
appl_all_processed_df_balanced['TARGET'].isnull().sum()))

In [ ]:
# Apply the following gc if memory is running slow
del appl_all_processed_df_balanced
gc.collect()

---

---

# Todo

Todo:
* cleaning:
    * num_df: normalize with z-score
* feature engineering:
    * make reciprocol, polynomial columns of the existing columns. 1/x, x^x.
    * multiplying each columns, two columns at a time.
    * asset items, income items, willingness(history + misc profile) items, loading(principle + interest) items
    * Integration from other tables?

https://ithelp.ithome.com.tw/articles/10202059
https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame
https://www.kaggle.com/parasjindal96/how-to-normalize-dataframe-pandas
    

---

## EDA

### Quick check for numerical columns

In [ ]:
numcol = df['CNT_FAM_MEMBERS']

In [ ]:
numcol.describe(), \
numcol.isnull().sum(), \
numcol.size

In [ ]:
numcol.value_counts(sort=True), numcol.unique().size

In [ ]:
# numcol_toYear = pd.to_numeric(\
#                               ((numcol.abs() / 365) \
#                                .round(0)) \
#                               ,downcast='integer')
# numcol_toYear.describe()

In [ ]:
# numcol_toYear.value_counts(sort=True), numcol_toYear.unique().size

### Quick check for categorical columns

In [ ]:
catcol = df['HOUR_APPR_PROCESS_START']

In [ ]:
catcol.unique(), \
catcol.unique().size

In [ ]:
catcol.value_counts(sort=True)

In [ ]:
catcol.isnull().sum(), \
catcol.size

In [ ]:
catcol.isnull().sum(), \
catcol.size

## Appendix

### Tool: Getting summary dataframe

In [ ]:
# might not be very useful at this point
def summary_df (data_df):
    """ 
    Output: a new dataframe with summary info from the input dataframe.
    Input: data_df, the original dataframe
    """
    summary_df = pd.concat([(data_df.describe(include='all')), \
           (data_df.dtypes.to_frame(name='dtypes').T), \
           (data_df.isnull().sum().to_frame(name='isnull').T), \
           (data_df.apply(lambda x:x.unique().size).to_frame(name='uniqAll').T)])
    return summary_df

def data_quality_df (data_df):
    """ 
    Output: a new dataframe with summary info from the input dataframe.
    Input: data_df, the original dataframe
    """
    data_quality_df = pd.concat([(data_df.describe(include='all')), \
           (data_df.dtypes.to_frame(name='dtypes').T), \
           (data_df.isnull().sum().to_frame(name='isnull').T), \
           (data_df.apply(lambda x:x.unique().size).to_frame(name='uniqAll').T)])
    return data_quality_df.iloc[[11,13,12,0,],:]


In [ ]:
data_quality_df(appl_all_df)

In [ ]:
# df.to_csv(file_name, encoding='utf-8', index=False)
# data_quality_df(df).to_csv("./eda_output/application_train_data_quality.csv")

In [ ]:
df['CNT_CHILDREN'].value_counts()

In [ ]:
df['CNT_CHILDREN'].value_counts().sum()

In [ ]:
df.describe()

In [ ]:
summary_df(df)

In [ ]:
# df.to_csv(file_name, encoding='utf-8', index=False)
# summary_df(df).to_csv("./eda_output/application_train_summary_df.csv")

---

### .nunique() function

In [ ]:
# nunique() function excludes NaN 
# i.e. it does not consider NaN as a "value", therefore NaN is not counted as a "unique value"
df.nunique()

In [ ]:
df.nunique() == df.apply(lambda x:x.unique().shape[0])

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].unique().shape[0]

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].nunique()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].unique().size

### .value_counts() function

In [ ]:
# .value_counts() function has similar viewpoint towards NaN.
# i.e. it does not consider null as a value, therefore not counted in .value_counts()

In [ ]:
df['NAME_TYPE_SUITE'].value_counts()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].isnull().sum()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].size

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].value_counts().sum() + df['AMT_REQ_CREDIT_BUREAU_YEAR'].isnull().sum() == \
df['AMT_REQ_CREDIT_BUREAU_YEAR'].size

### 重複值

In [ ]:
# Counting unique values (cf. .nunique() function, see above section)
# This code was retrieved from HT

df.apply(lambda x:x.unique().shape[0])

In [ ]:
# It is the same if you write (df.apply(lambda x:x.unique().size))
assert (df.apply(lambda x:x.unique().shape[0])==df.apply(lambda x:x.unique().size)).all

In [ ]:
# # %timeit showed the performances are similar
# %timeit df.apply(lambda x:x.unique().shape[0])
# %timeit df.apply(lambda x:x.unique().size)

### 空值

In [ ]:
# 含空值欄位占比
print(f"{df.isnull().any().sum()} in {df.shape[1]} columns (ratio: {(df.isnull().any().sum()/df.shape[1]).round(2)}) has empty value(s)")


---

## re-casting to reduce memory use (beta)

In [ ]:
# np.isfinite(num_df).all().value_counts()

In [ ]:
# num_df_finite = num_df[num_df.columns[np.isfinite(num_df).all()]]
# num_df_infinite = num_df[num_df.columns[np.isfinite(num_df).all() == False]]
# num_df_finite.shape, num_df_infinite.shape

In [ ]:
# assert num_df_finite.shape[0] == num_df_infinite.shape[0] == num_df.shape[0]
# assert num_df_finite.shape[1] + num_df_infinite.shape[1] == num_df.shape[1]

In [ ]:
# def reduce_mem_usage(props, finite:bool = True):
#     props.info(verbose=False)
#     start_mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage of properties dataframe is :",start_mem_usg," MB")
#     if finite == True:  
#         props[props.columns[(props.min()>=0) & (props.max()<255)]] = \
#         props[props.columns[(props.min()>=0) & (props.max()<255)]].astype(np.uint8, copy=False)
#         props.info(verbose=False)

#         props[props.columns[(props.min()>=0) &(props.max() >= 255) & (props.max()<65535)]] = \
#         props[props.columns[(props.min()>=0) &(props.max() >= 255) & (props.max()<65535)]] \
#         .astype(np.uint16, copy=False)
#         props.info(verbose=False)

#         props[props.columns[(props.min()>=0) &(props.max() >= 65535) & (props.max()<4294967295)]] = \
#         props[props.columns[(props.min()>=0) &(props.max() >= 65535) & (props.max()<4294967295)]] \
#         .astype(np.uint32, copy=False)
#         props.info(verbose=False)

#         props[props.columns[(props.min()>=0) &(props.max() >= 4294967295)]] = \
#         props[props.columns[(props.min()>=0) &(props.max() >= 4294967295)]] \
#         .astype(np.uint64, copy=False)
#         props.info(verbose=False)
#     else:
#         props = props.astype(np.float32, copy=False)
#         props.info(verbose=False)
        
#     print("___MEMORY USAGE AFTER COMPLETION:___")
#     mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage is: ",mem_usg," MB")
#     print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    
#     return props

# if num_na_df_finite.min()>=0:
#     if num_na_df_finite.max() < 255:
#         props[col] = props[col].astype(np.uint8)
#     elif num_na_df_finite.max() < 65535:
#         props[col] = props[col].astype(np.uint16)
#     elif num_na_df_finite.max() < 4294967295:
#         props[col] = props[col].astype(np.uint32)
#     else:
#         props[col] = props[col].astype(np.uint64)

In [ ]:
# num_df_finite.info()

In [ ]:
# num_df_finite = reduce_mem_usage(num_df_finite, finite = True)

In [ ]:
# num_df_infinite.info()

In [ ]:
# num_df_infinite = reduce_mem_usage(num_df_infinite, finite = False)

In [ ]:
# num_df = pd.concat([num_df_finite, num_df_infinite], axis ='columns')
# num_df.info()

In [ ]:
# assert num_df_finite.shape[0] == num_df_infinite.shape[0] == num_df.shape[0]
# assert num_df_finite.shape[1] + num_df_infinite.shape[1] == num_df.shape[1]

In [ ]:
# del num_df_finite
# del num_df_infinite
# gc.collect()

THE END